In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torchvision.io import read_image
import torchvision.transforms.v2 as transformsv2
import sys
torch.manual_seed(42)

sys.path.append("..")

RESNET18_HEIGHT = 224
RESNET18_WIDTH = 224

class RFW(Dataset):

    def __init__(self, img_path, attr_path, transforms, png):

        self.attr = pd.read_csv(attr_path).to_numpy()
        print(f'attr: {self.attr}')
        self.img_path = img_path
        self.transforms = transforms
        self.png = png
        
    def __len__(self):
        return len(self.attr)
    
    def __getitem__(self, idx):

        if self.png:
            img =  read_image(os.path.join(self.img_path, self.attr[idx][2], self.attr[idx][1]))
        else:
            img =  read_image(os.path.join(self.img_path, self.attr[idx][2], self.attr[idx][1].replace("png", "jpg")))
        return self.transforms(img), torch.from_numpy(self.attr[idx][3:].astype(np.float32)),\
            self.attr[idx][2].split("/")[0], f'{self.attr[idx][2]}/{self.attr[idx][1]}'



def create_dataloaders(
    img_path, 
    attr_path, 
    batch_size, 
    train_test_ratio=0.7, 
    png=True, 
    seed=42
):

    tfs = transformsv2.Compose([
        transformsv2.Resize((RESNET18_HEIGHT, RESNET18_WIDTH)),
        transformsv2.ToTensor()
    ])

    # Create Dataset
    data = RFW(img_path, attr_path, tfs, png)

    generator = torch.Generator().manual_seed(seed)

    trainset_size = int(len(data) * train_test_ratio)
    validaset_size = int((len(data) - trainset_size) * 0.5)
    testset_size = len(data) - trainset_size - validaset_size

    trainset, valset, testset = random_split(data, [trainset_size, validaset_size, testset_size], generator)

    # Create data loaders
    train_loader = DataLoader(trainset, batch_size, shuffle=True)
    val_loader = DataLoader(valset, batch_size)
    test_loader = DataLoader(testset, batch_size)

    return train_loader, val_loader, test_loader

In [ ]:
RACE_LABELS = ['Indian', 'Asian', 'African', 'Caucasian']
ROOT = '/media/global_data/fair_neural_compression_data/decoded_rfw/decoded_64x64'
RFW_LABELS_DIR = "/media/global_data/fair_neural_compression_data/datasets/RFW/clean_metadata/numerical_labels_sorted.csv"
RATIO = 0.8
BATCH_SIZE = 32
model_names = ['cheng2020-attn', 'hyperprior', 'mbt2018', 'qarv', 'qres17m']
# attribites = ['']
# hard coding model_name to be the cheng2020-attn model
model_name = 'cheng2020-attn'
datasets_names = ['fairface', 'celebA']
dataset_name = 'celebA'

qualities = ['q_0001', 'q_0009', 'q_1', 'q_2', 'q_3']
# mapping model name to corresponding decoded image storage dir.
qualities_dict = {'hyperprior':['q_0001', 'q_0009', 'q_1', 'q_2', 'q_3'], 
                  'mbt2018':['q_0001', 'q_0009', 'q_1', 'q_2', 'q_3'], 
                  'cheng2020-attn':['q_0001', 'q_0009', 'q_1', 'q_2', 'q_3'], 
                  'qres17m':['1', '3', '6', '9', '12'], 
                  'qarv':['lmb_1', 'lmb_4', 'lmb_8', 'lmb_16', 'lmb_32']}
model_name_dict = {'hyperprior':'hyperprior', 
                  'mbt2018':'mbt2018', 
                  'cheng2020-attn':'cheng2020-attn', 
                  'qres17m':'qres17m_lmb_64', 
                  'qarv':'qarv'}

for quality in qualities_dict[model_name]:
    train_loader, valid_loader, test_loader = create_dataloaders(
        f'{ROOT}/{model_name_dict[model_name]}/{dataset_name}/{quality}', 
        RFW_LABELS_DIR, 
        BATCH_SIZE, 
        RATIO
    )

In [ ]:
test_file_names = []
for batch in test_loader:
    test_file_names.extend(list(batch[-1]))
# test_file_names

In [ ]:
from tqdm import tqdm

ATTRIBUTE_INDECIES = {
    'skin_type': 0,
    'lip_type': 1,
    'nose_type': 2,
    'eye_type': 3,
    'hair_type': 4,
    'hair_color': 5
}

def save_race_based_predictions(
        models,  
        dataloader, 
        device, 
        prediction_save_dir,
        attributes
    ):
    all_predictions = {'Indian': {attr: torch.tensor([]) for attr in attributes}, 
                       'Caucasian': {attr: torch.tensor([]) for attr in attributes}, 
                       'Asian': {attr: torch.tensor([]) for attr in attributes},  
                       'African': {attr: torch.tensor([]) for attr in attributes}}
    all_labels = {'Indian': {attr: torch.tensor([]) for attr in attributes}, 
                  'Caucasian': {attr: torch.tensor([]) for attr in attributes}, 
                  'Asian': {attr: torch.tensor([]) for attr in attributes}, 
                  'African': {attr: torch.tensor([]) for attr in attributes}}
    all_file_names = {
        'Indian': {attr: [] for attr in attributes}, 
        'Caucasian': {attr: [] for attr in attributes}, 
        'Asian': {attr: [] for attr in attributes},  
        'African': {attr: [] for attr in attributes}
    }
    print(f'prediction_save_dir: {prediction_save_dir}')
    dataloader = tqdm(dataloader, desc="Getting Predictions", unit="batch")
    with torch.no_grad():
        for j, model in enumerate(models):
            model.eval()
            for _, data in enumerate(dataloader):
                inputs, labels, race, file_names = data
                file_names = np.array(list(file_names))
                race = np.array(race)

                inputs = inputs.to(torch.float).to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                for i, (head, predictions) in enumerate(outputs.items()):
                    head_preds = predictions.argmax(dim=1).cpu()

                    for race_label in all_labels:
                        race_indices = np.array((race == race_label).nonzero()[0])
                        race_predictions = head_preds[race_indices]
                        race_file_names = file_names[race_indices]
                        race_labels = labels[:, ATTRIBUTE_INDECIES[head]][race_indices]
                    
                        all_predictions[race_label][head] = torch.cat((all_predictions[race_label][head], race_predictions.to('cpu')), dim=0)
                        all_labels[race_label][head] = torch.cat((all_labels[race_label][head], race_labels.to('cpu')), dim=0)
                        all_file_names[race_label][head].extend(list(race_file_names))
    # with open(prediction_save_dir + '/sep_predictions.pkl', 'wb+') as f:
    #     pickle.dump(all_predictions, f)
    # with open(prediction_save_dir + '/sep_labels.pkl', 'wb+') as f:
    #     pickle.dump(all_labels, f)


    return all_predictions, all_labels, all_file_names

In [ ]:
import pickle
from multi_head_resnet import MultiHeadResNet
all_predictions = {}
all_labels = {}
all_file_names = {}
attributes = ['eye_type', 'hair_color', 'hair_type', 'nose_type', 'skin_type']
# attributes = ['eye_type']
PRED_ROOT = '/media/global_data/fair_neural_compression_data/pred_with_raw_model_debug'# new folder 
# same <models> list for all the compression models.
models = []
for attribute in attributes:
    # this dir is hardcoded as we store clean model training results here. 
    attribute_model_path = f'{PRED_ROOT}/hyperprior/celebA/clean/{attribute}_best.pth'
    models.append(torch.load(attribute_model_path))


# this is True if want to redo the evaluation. If False, we are loading from previous runs to save time. 
redo_evaluation = False

if redo_evaluation:
    for model_name in model_names:
        print(model_name)
        all_predictions[model_name] = {}
        all_labels[model_name] = {}
        all_file_names[model_name] = {}
        for quality in qualities_dict[model_name]:
            train_loader, valid_loader, test_loader = create_dataloaders(
                f'{ROOT}/{model_name_dict[model_name]}/{dataset_name}/{quality}', 
                RFW_LABELS_DIR, 
                BATCH_SIZE, 
                RATIO
            )
            print(quality)
            all_predictions[model_name][quality], \
            all_labels[model_name][quality], \
            all_file_names[model_name][quality] = \
                save_race_based_predictions(
                    models,  
                    test_loader, 
                    'cuda:1', 
                    "",
                    attributes
                )      
else:
    # load from saved files
    # already have files, so load, instead of re-run the eval. 
    file = open("./raw_model_predictions.pkl",'rb')
    all_predictions = pickle.load(file)
    file = open("./raw_model_labels.pkl",'rb')
    all_labels = pickle.load(file)
    file = open("./raw_model_filenames.pkl",'rb')
    all_file_names = pickle.load(file)

save evaluation results. Uncomment when need to save evaluation results.

In [ ]:
# save prediction results
# with open('./raw_model_predictions.pkl', 'wb+') as f:
#     pickle.dump(all_predictions, f)
# with open('./raw_model_labels.pkl', 'wb+') as f:
#     pickle.dump(all_labels, f)
# with open('./raw_model_filenames.pkl', 'wb+') as f:
#     pickle.dump(all_file_names, f)

In [ ]:
cheng_preds, cheng_labels = all_predictions['cheng2020-attn'], all_labels['cheng2020-attn']

In [ ]:
cheng_file_names = all_file_names['cheng2020-attn']

In [ ]:
wrong_preds_file_names = {}

for quality in qualities:
    wrong_preds_file_names[quality] = {}
    for race in RACE_LABELS:
        wrong_preds_file_names[quality][race] = {}
        for attribute in attributes:
            mask = torch.ne(torch.tensor(cheng_preds[quality][race][attribute]), torch.tensor(cheng_labels[quality][race][attribute]))
            wrong_preds_file_names[quality][race][attribute] = \
                np.array(cheng_file_names[quality][race][attribute])[np.array(torch.nonzero(mask).squeeze())]

In [ ]:
import json
data_path = '/media/global_data/fair_neural_compression_data/decoded_rfw/decoded_64x64'
bpp_data = {}
datasets = ['celebA', 'fairface']
for model_name in ['cheng2020-attn']:
    if model_name == 'jpeg':
        continue
    print(f'model_name: {model_name}')
    bpp_data[model_name] = {}
    model_path = f'{data_path}/{model_name}'
    for dataset in datasets:
        print(f'dataset: {dataset}')
        bpp_data[model_name][dataset] = {}
        dataset_path = f'{model_path}/{dataset}'
        for quality in qualities:
            stats_path = f'{dataset_path}/{quality}/stats.json'
            with open(stats_path, 'r') as json_file:
                stats_data = json.load(json_file)
            if "results" in stats_data:
                bpp_data[model_name][dataset][quality] = stats_data['results']['bpp']
            elif "est_bpp" in stats_data:
                bpp_data[model_name][dataset][quality] = stats_data['est_bpp']

In [ ]:
# Tian: inspect African classification results
from sklearn.metrics import confusion_matrix
import random
random.seed(42)
merged_skin_type={
        'African':([4, 5], 3), 
        'Asian':([3, 2], 1), 
        'Caucasian':([3, 2], 1), 
        'Indian':([4,3], 2)
}
race_of_interest = 'African'
attribute_of_interest = 'skin_type'

for quality in qualities:
    # print(quality)
    merged_preds = cheng_preds[quality][race_of_interest][attribute_of_interest].clone()
    merged_labels = cheng_labels[quality][race_of_interest][attribute_of_interest].clone()
    if attribute_of_interest =='skin_type':
        for target in merged_skin_type[race_of_interest][0]:
            merged_preds[merged_preds==target] = merged_skin_type[race_of_interest][1]
            merged_labels[merged_labels==target] = merged_skin_type[race_of_interest][1]
        class_3_indices = set(np.array((merged_labels==3).nonzero().squeeze()))
        predict_2_indices = set(np.array((merged_preds==2).nonzero().squeeze()))
        intersection_indices = list(class_3_indices.intersection(predict_2_indices))
        intersection_filenames = np.array(cheng_file_names[quality][race_of_interest][attribute_of_interest])[intersection_indices]
        intersection_filenames = list(intersection_filenames)
        # print(len(class_3_indices))
        # print(len(predict_2_indices))
        # print(len(intersection_indices))

selected_intersection_filenames = random.sample(intersection_filenames, 10)
print(selected_intersection_filenames)
    # print(confusion_matrix(cheng_labels[quality][race_of_interest][attribute_of_interest], cheng_preds[quality][race_of_interest][attribute_of_interest],labels=[0, 1, 2, 3, 4, 5]))
    # print(confusion_matrix(merged_labels, merged_preds,labels=[0, 1, 2, 3, 4, 5]))

# print(len(cheng_preds[quality][race_of_interest][attribute_of_interest]))

# for quality in qualities:
#     print(len(wrong_preds_file_names[quality][race_of_interest][attribute_of_interest]))

In [ ]:
# plot mis-classified images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
RFW_ROOT = '/media/global_data/fair_neural_compression_data/datasets/RFW/data_64'
fig, axes = plt.subplots(10, 6, figsize=(8, 12))
plt.setp(axes, xticks=[], yticks=[], frame_on=False) # remove black borders, no xy axis
for i, file_name in enumerate(selected_intersection_filenames):
    for j in range(len(qualities) + 1):
        if j == len(qualities):
            image_path = f'{RFW_ROOT}/{file_name}'
        else:
            image_path = f'{ROOT}/{"cheng2020-attn"}/{dataset_name}/{qualities[j]}/{file_name}'
        img = mpimg.imread(image_path)
        axes[i][j].imshow(img)
plt.savefig(f'../../plots/pred_with_raw_model/{model_name}_African_skin_type_3_to_2_errors.pdf', dpi=300, bbox_inches='tight')